In [126]:
import pandas as pd
import numpy as np

In [127]:
sched_df = pd.read_csv('./2022_schedule.csv')

Will have to make some code here to handle the original file format if use an updated version  
Think I manually dropped the first week 1 - 11 games in excel  
Also deleted some columns 
Manually gave the home/away column name - OG data has no column name for it

### Create home and away team columns from data
Data comes in with winner and loser teams vs home and away  
Have to use the home/away column

In [128]:
sched_df['home'] = np.where(
    sched_df['home/away'] == '@', 
    sched_df['Loser/tie'], 
    sched_df['Winner/tie']
    )
sched_df['away'] = np.where(
    sched_df['home/away'] == '@', 
    sched_df['Winner/tie'], 
    sched_df['Loser/tie']
    )


### Check if any home and away teams are the same value
If ```True in Same``` returns True than have an issue 

In [129]:
same = np.where(sched_df['home'] == sched_df['away'], True, False)
True in same

False

### Remove City from Team Name
Also convert to all lowercase

In [130]:

def get_short_team_name(full_team_name):
    short_name = full_team_name.rsplit(None, 1)[-1]
    lc_short_name = short_name.lower()
    return lc_short_name


In [131]:
sched_df['home'] = sched_df['home'].apply(lambda x: get_short_team_name(x))
sched_df['away'] = sched_df['away'].apply(lambda x: get_short_team_name(x))

### Get Home and Away Score 
Stored in original file as Points Winner (PtsW) and Points Loser (PtsL)

In [132]:
sched_df['home_score'] = np.where(sched_df['home/away'] == '@', sched_df['PtsL'], sched_df['PtsW'])
sched_df['away_score'] = np.where(sched_df['home/away'] == '@', sched_df['PtsW'], sched_df['PtsL'])


#### Drop unused columns

In [133]:
sched_df = sched_df.drop(columns=['Winner/tie','home/away','Loser/tie','PtsW','PtsL'])


### [datetime docs]('https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html')

## Create Game Time Column based on date and time columns

In [134]:
sched_df['GameTime'] = pd.to_datetime(sched_df['Date'] + 'T' + sched_df['Time'] + '-0500', utc=True )
print(sched_df.dtypes)
sched_df.head()

Week                        int64
Day                        object
Date                       object
Time                       object
home                       object
away                       object
home_score                float64
away_score                float64
GameTime      datetime64[ns, UTC]
dtype: object


,Week,Day,Date,Time,home,away,home_score,away_score,GameTime
0,12,Thu,2022-11-24,12:30PM,lions,bills,25.0,28.0,2022-11-24 17:30:00+00:00
1,12,Thu,2022-11-24,4:30PM,cowboys,giants,28.0,20.0,2022-11-24 21:30:00+00:00
2,12,Thu,2022-11-24,8:20PM,vikings,patriots,33.0,26.0,2022-11-25 01:20:00+00:00
3,12,Sun,2022-11-27,1:00PM,titans,bengals,16.0,20.0,2022-11-27 18:00:00+00:00
4,12,Sun,2022-11-27,1:00PM,browns,buccaneers,23.0,17.0,2022-11-27 18:00:00+00:00


### Create Start and End Times based off Game Time
Start time is 4 days 1 hour before start of game  
End time is 1 hour before start of game

In [135]:
sched_df['SearchStartTime'] = sched_df['GameTime'] - pd.to_timedelta('4 days 01:00:00')
sched_df['SearchEndTime'] = sched_df['GameTime'] - pd.to_timedelta('01:00:00')
sched_df.head()

,Week,Day,Date,Time,home,away,home_score,away_score,GameTime,SearchStartTime,SearchEndTime
0,12,Thu,2022-11-24,12:30PM,lions,bills,25.0,28.0,2022-11-24 17:30:00+00:00,2022-11-20 16:30:00+00:00,2022-11-24 16:30:00+00:00
1,12,Thu,2022-11-24,4:30PM,cowboys,giants,28.0,20.0,2022-11-24 21:30:00+00:00,2022-11-20 20:30:00+00:00,2022-11-24 20:30:00+00:00
2,12,Thu,2022-11-24,8:20PM,vikings,patriots,33.0,26.0,2022-11-25 01:20:00+00:00,2022-11-21 00:20:00+00:00,2022-11-25 00:20:00+00:00
3,12,Sun,2022-11-27,1:00PM,titans,bengals,16.0,20.0,2022-11-27 18:00:00+00:00,2022-11-23 17:00:00+00:00,2022-11-27 17:00:00+00:00
4,12,Sun,2022-11-27,1:00PM,browns,buccaneers,23.0,17.0,2022-11-27 18:00:00+00:00,2022-11-23 17:00:00+00:00,2022-11-27 17:00:00+00:00


### Save to CSV

In [136]:

sched_df.to_csv('./2022_schedule_cleaned.csv', index=False)